In [4]:
!pip install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q -U transformers datasets peft bitsandbytes accelerate evaluate rouge_score bert_score sacrebleu

In [1]:
# Install all necessary libraries
# !pip install -q -U transformers datasets peft torch evaluate rouge_score bert_score sacrebleu

# Import all necessary libraries
from datasets import load_from_disk
from peft import PeftModel
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    pipeline,
    BitsAndBytesConfig
)
from google.colab import drive
import pandas as pd
import torch
import evaluate
from tqdm import tqdm

# Set pandas display option to see full text
pd.set_option('display.max_colwidth', None)

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# --- 2. Load Both Trained Models ---
print("Loading models...")
ranker_path = "/content/drive/MyDrive/deberta_ranker_final_large_run/checkpoint-75"
ranker_model = AutoModelForSequenceClassification.from_pretrained(ranker_path)
ranker_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
ranker_pipe = pipeline("text-classification", model=ranker_model, tokenizer=ranker_tokenizer, device=0)

gemma_base_model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
gemma_base_model = AutoModelForCausalLM.from_pretrained(gemma_base_model_id, quantization_config=bnb_config, device_map="auto")
gemma_tokenizer = AutoTokenizer.from_pretrained(gemma_base_model_id)
gemma_adapter_path = "/content/drive/MyDrive/gemma_summarizer_run/checkpoint-125"
gemma_model = PeftModel.from_pretrained(gemma_base_model, gemma_adapter_path)
print("Models loaded.")

# --- 3. Run the Error Analysis Pipeline ---
print("Loading data and starting error analysis...")
full_dataset = load_from_disk("/content/drive/MyDrive/cnn_dailymail_dataset")
test_slice = full_dataset["test"].select(range(200))
bertscore = evaluate.load("bertscore")
error_count = 0

for i, example in enumerate(tqdm(test_slice)):
    article = example['article']
    human_summary = example['highlights']

    # Step A: Generate candidate summaries
    prompt = f"### Instruction:\nSummarize the following news article.\n\n### Input:\n{article}\n\n### Response:\n"
    input_ids = gemma_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = gemma_model.generate(**input_ids, max_new_tokens=100, do_sample=True, top_k=50, num_return_sequences=3)

    candidate_summaries = []
    for j in range(3):
        generated_text = gemma_tokenizer.decode(outputs[j], skip_special_tokens=True)
        candidate_summaries.append(generated_text[len(prompt):].strip())

    # Step B: Rank the candidates
    formatted_for_ranker = [f"summarize: {article[:4000]} <sep> candidate: {cand}" for cand in candidate_summaries]
    ranking_predictions = ranker_pipe(formatted_for_ranker, top_k=None, truncation=True)

    # Step C: Calculate BERTScore for EACH candidate and find the best summary
    scores = bertscore.compute(predictions=candidate_summaries, references=[human_summary] * 3, lang="en")

    best_candidate_idx = scores['f1'].index(max(scores['f1']))
    ranker_choice_idx = -1
    highest_ranker_score = -1

    for k, result_pairs in enumerate(ranking_predictions):
        score_for_label_1 = 0
        for pair in result_pairs:
            if pair['label'] == 'LABEL_1':
                score_for_label_1 = pair['score']
                break
        if score_for_label_1 > highest_ranker_score:
            highest_ranker_score = score_for_label_1
            ranker_choice_idx = k

    # Step D: Check for errors and print
    if ranker_choice_idx != best_candidate_idx:
        error_count += 1
        print(f"\n\n--- RANKING ERROR FOUND (Example {i+1}) ---")
        print(f"\nHUMAN SUMMARY: {human_summary}")

        # Create a detailed DataFrame for analysis
        results_df = pd.DataFrame({
            "candidate_summary": candidate_summaries,
            "bertscore_f1": scores['f1'],
            "ranker_score": [p[0]['score'] if p[0]['label'] == 'LABEL_1' else p[1]['score'] for p in ranking_predictions]
        })
        results_df = results_df.sort_values(by="ranker_score", ascending=False)
        print("\nMODEL RANKING:")
        print(results_df)
        print("-----------------------------------------")

print(f"\n--- Error Analysis Complete ---")
print(f"Found {error_count} ranking errors out of {len(test_slice)} examples.")

Mounting Google Drive...
Mounted at /content/drive
Loading models...


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


...



--- RANKING ERROR FOUND (Example 197) ---

HUMAN SUMMARY: ISIS raided Yarmouk refugee camp near Damascus on April 1 .
18,000 Palestinian refugees remain in the camp, cut off from vital aid .
One defiant elderly resident told CNN: "I will not leave the Yarmouk camp even if I am 75 or 76 years old"

MODEL RANKING:
                                                                                                                                                                                                                                                                                                                                                                  candidate_summary  \
0                                                                         Many Palestinians live in Syria , as it was the region first settled by Jews .\nThe UN says the country has suffered war-related damage to some 15 million homes\nISIS took over an area near the Syrian enclave of Yarmouk  \nThe camp was

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 99%|█████████▉| 198/200 [20:28<00:12,  6.44s/it]



--- RANKING ERROR FOUND (Example 198) ---

HUMAN SUMMARY: MTV Movie Awards host Amy Schumer had some hits and misses during the show .
Rebel Wilson tossed in a censor-worthy joke .

MODEL RANKING:
                                                                                                                                                                                                                                                                                                                candidate_summary  \
0                                                                                  J-Lo's 'Boyhood'/HPV reference caused a stir .\nAmy Schumer's 'Pitch Perfect' set was a sweet treat\nKimmel joked about Kevin Hart not being the tallest, short jokes ... but was a bit of an oddball\nSee all 86 MTV Movie Awards photos here   
1  2015 MTV Movie Awards highlights included Jennifer Lopez's first ever twerking moment .\nAmy Schumer's 'Girls Just Wanna Ride' joke nearly resulted 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 200/200 [20:43<00:00,  6.22s/it]


--- Error Analysis Complete ---
Found 132 ranking errors out of 200 examples.


In [2]:
# Import all necessary libraries
import pickle
import pandas as pd
from datasets import load_from_disk
from peft import PeftModel
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    pipeline,
    BitsAndBytesConfig
)
from google.colab import drive
import torch
import evaluate
from tqdm import tqdm

# Set pandas display option to see full text
pd.set_option('display.max_colwidth', None)

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# --- 2. Load Both Trained Models ---
print("Loading models...")
ranker_path = "/content/drive/MyDrive/deberta_ranker_final_large_run/checkpoint-75"
ranker_model = AutoModelForSequenceClassification.from_pretrained(ranker_path)
ranker_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
ranker_pipe = pipeline("text-classification", model=ranker_model, tokenizer=ranker_tokenizer, device=0)

gemma_base_model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
gemma_base_model = AutoModelForCausalLM.from_pretrained(gemma_base_model_id, quantization_config=bnb_config, device_map="auto")
gemma_tokenizer = AutoTokenizer.from_pretrained(gemma_base_model_id)
gemma_adapter_path = "/content/drive/MyDrive/gemma_summarizer_run/checkpoint-125"
gemma_model = PeftModel.from_pretrained(gemma_base_model, gemma_adapter_path)
print("Models loaded.")

# --- 3. Run the Error Analysis Pipeline ---
print("Loading data and starting 'Worst-Case' error analysis...")
full_dataset = load_from_disk("/content/drive/MyDrive/cnn_dailymail_dataset")
test_slice = full_dataset["test"].select(range(200))
bertscore = evaluate.load("bertscore")
error_count = 0

for i, example in enumerate(tqdm(test_slice)):
    article = example['article']
    human_summary = example['highlights']

    # Step A: Generate candidate summaries
    prompt = f"### Instruction:\nSummarize the following news article.\n\n### Input:\n{article}\n\n### Response:\n"
    input_ids = gemma_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = gemma_model.generate(**input_ids, max_new_tokens=100, do_sample=True, top_k=50, num_return_sequences=3)

    candidate_summaries = []
    for j in range(3):
        generated_text = gemma_tokenizer.decode(outputs[j], skip_special_tokens=True)
        candidate_summaries.append(generated_text[len(prompt):].strip())

    # Step B: Rank the candidates
    formatted_for_ranker = [f"summarize: {article[:4000]} <sep> candidate: {cand}" for cand in candidate_summaries]
    ranking_predictions = ranker_pipe(formatted_for_ranker, top_k=None, truncation=True)

    # Step C: Calculate BERTScore for EACH candidate
    scores = bertscore.compute(predictions=candidate_summaries, references=[human_summary] * 3, lang="en")

    worst_candidate_idx = scores['f1'].index(min(scores['f1']))
    ranker_choice_idx = -1
    highest_ranker_score = -1

    for k, result_pairs in enumerate(ranking_predictions):
        score_for_label_1 = 0
        for pair in result_pairs:
            if pair['label'] == 'LABEL_1':
                score_for_label_1 = pair['score']
                break
        if score_for_label_1 > highest_ranker_score:
            highest_ranker_score = score_for_label_1
            ranker_choice_idx = k

    # Step D: Check for errors and print
    if ranker_choice_idx == worst_candidate_idx:
        error_count += 1
        print(f"\n\n--- WORST-CASE RANKING ERROR FOUND (Example {i+1}) ---")
        print(f"\nHUMAN SUMMARY: {human_summary}")

        results_df = pd.DataFrame({
            "candidate_summary": candidate_summaries,
            "bertscore_f1": scores['f1'],
            "ranker_score": [p[0]['score'] if p[0]['label'] == 'LABEL_1' else p[1]['score'] for p in ranking_predictions]
        })
        results_df = results_df.sort_values(by="ranker_score", ascending=False)
        print("\nMODEL RANKING:")
        print(results_df)
        print("-----------------------------------------")

print(f"\n--- Worst-Case Error Analysis Complete ---")
print(f"Found {error_count} worst-case ranking errors out of {len(test_slice)} examples.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading models...


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Models loaded.
Loading data and starting 'Worst-Case' error analysis...


  0%|          | 0/200 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▎         | 5/200 [00:30<18:29,  5.69s/it]

...



--- WORST-CASE RANKING ERROR FOUND (Example 190) ---

HUMAN SUMMARY: Dean Obeidallah: Apple's new emoji lineup is diverse in race, ethnicity and sexual orientation .
It's just like America, but what took Apple so long? Obeidallah asks .
He says change may rankle (or win over?) conservatives who discriminate against people because of their sexual orientation .

MODEL RANKING:
                                                                                                                                                                                                                             candidate_summary  \
2                 Apple gives the world emoji it says has now included a variety of skin tones .\nThe new design comes a day after controversy over 'Santa's' background .\nThe new emojis don't include Muslim Santa, gay couple and redheads   
0                                      Apple introduces new emoji with diverse range of skin tones .\nUsers must select skin tone before

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 96%|█████████▋| 193/200 [19:58<00:46,  6.66s/it]



--- WORST-CASE RANKING ERROR FOUND (Example 193) ---

HUMAN SUMMARY: Pentagon releases map showing coalition forces have taken back 25-30% of Iraq territory from ISIS .
Counterinsurgency specialist Afzal Ashraf on what new data tells us about fight .
Ashraf: Where it counts (ISIS is) not standing and fighting .

MODEL RANKING:
                                                                                                                                                                                                                                                                                                                                                                                                                                                               candidate_summary  \
1                                                                                                                                                                                                        

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 200/200 [20:47<00:00,  6.24s/it]


--- Worst-Case Error Analysis Complete ---
Found 58 worst-case ranking errors out of 200 examples.
